# Making fake data

For the sake of this workshop, we'll be using
some fake data from a fake experiment. This Jupyter
notebook contains code to create the data.

Let's say that we're interested in seeing how people bond
(or build rapport) with one another based on different
kinds of conversations. In our fake experiment, we'll ask
two strangers to come into the lab and ask them to have
a conversation with one another. They'll be randomly 
assigned to either have a conversation about a really good thing
that happened to them in the past week ("celebration"
conversation) or a really bad thing that happened to them in 
the past week ("commiseration" conversation). 
Their conversation will be audio- and video-recorded.

Let's say that this fake experiment is going to test
the similarity of positive emotion words versus negative
emotion words that people said during those
conversations. We want to see whether partners who use
similar negative and/or positive words in these two
different kinds of conversations are rated as having
higher rapport by expert observers.

To figure out negative and positive word similarity,
let's say that we transcribed their conversations and
simply counted up the number of times per turn that
each person used negative words and positive words.
We then get the similarity score by taking the correlation
of those two scores.

To track rapport, let's say that we recruited 2
expert observers as raters. Let's say that we trained them
to continuously rate rapport from the videos using
a joystick-style method (cf. Sadler, Ethier, Gunn,
Duong, Woody, 2009), creating a time series of ratings between
0 and 1. We then obtained a single rapport rating for the dyad
by taking the mean of the time series.

This means our data will look something like this:
* **Dependent variable**: `rapport` (0 to 1; average of 
    continuous moment-to-moment rating by expert observers)
* **Independent variables**: 
    * `negative_word_similarity` (-1 to +1; correlation 
        of negative words used by conversation partners)
    * `positive_word_similarity` (-1 to +1; correlation 
        of positive words used by conversation partners)
    * `conversation_type` (0 = "commiseration" conversation;
        1 = "celebration" conversation)

**Created by**: A. Paxton (University of 
Connecticut)

**Last modified**: 10 July 2020

***

## Import libraries

When programming, there are a lot of basic functions (or 
specific actions that are applied to data; e.g., addition,
subtraction) that are built into the core of the language. 
To expand the power of programming languages, people often
write and share additional *libraries* or *packages* that
have been developed to incorporate other functions. We
have to `import` each additional package or library before
we can use those functions, which we do below.

We can import entire libraries:

In [ ]:
import random

We can assign shorter names to whole libraries
to make typing easier:

In [ ]:
import pandas as pd
import numpy as np

And we can even just grab specific functions if we
don't need the whole package:

In [ ]:
from sklearn.preprocessing import scale

## Specify how many dyads we want

How many total dyads do we want?

In [ ]:
dyad_n = 40

How many commiseration dyads do we want?

In [ ]:
commiseration_dyads = 25

However many we have left over is how many
celebration dyads we have.

In [ ]:
celebration_dyads = dyad_n - commiseration_dyads

## Add dyads to dataframe

Create dyad IDs and dataframe for commiseration condition.

In [ ]:
commiseration_dyad_IDs = range(1, 
                               commiseration_dyads+1)

In [ ]:
commiseration_dyad_df = pd.DataFrame({'dyad_ID': commiseration_dyad_IDs,
                                      'condition': 0})

Create dyad IDs and dataframe for celebration condition.

In [ ]:
celebration_dyad_IDs = range(commiseration_dyads+1,
                             dyad_n+1)

In [ ]:
celebration_dyad_df = pd.DataFrame({'dyad_ID': celebration_dyad_IDs,
                                   'condition': 1})

## Randomly generate rapport variables

Whenever we rely on random number generation in programming, it's
critical to set a seed for our random number generator. This allows
us to facilitate *computational reproducibility*, or the ability for
us to re-run the code and get precisely the same values. In the
case of randomly generated numbers or any programs that incorporate
randomness, it's even more important to set the seed.

In [ ]:
random.seed(30)

Let's make sure that our commiseration mean 
is higher than the celebration mean.

In [ ]:
commiseration_mean = .6

In [ ]:
commiseration_sigma = .2

In [ ]:
celebration_mean = .3

In [ ]:
celebration_sigma = .1

Now, let's create separate distributions for each rapport.

In [ ]:
commiseration_rapport = [round(np.random.normal(commiseration_mean,
                                                commiseration_sigma), 1)
                         for dyad in range(0, commiseration_dyads)]

In [ ]:
celebration_rapport = [round(np.random.normal(celebration_mean,
                                              celebration_sigma), 1)
                         for dyad in range(0, celebration_dyads)]

And now we add them to the dataframes.

In [ ]:
commiseration_dyad_df['rapport'] = commiseration_rapport

In [ ]:
celebration_dyad_df['rapport'] = celebration_rapport

## Randomly create similarity scores

Let's create them for the commiseration participants.

In [ ]:
commiseration_positive = commiseration_dyad_df['rapport']/5 * .25 + [
        random.random() for dyad in range(0, commiseration_dyads)]

In [ ]:
commiseration_negative = commiseration_dyad_df['rapport']/5 * .75 + [
            random.random() for dyad in range(0, commiseration_dyads)]

And for the celebration participants.

In [ ]:
celebration_positive = celebration_dyad_df['rapport']/5 * .5 + [
        random.random() for dyad in range(0, celebration_dyads)]

In [ ]:
celebration_negative = celebration_dyad_df['rapport']/5 * .3 + [
        random.random() for dyad in range(0, celebration_dyads)]

Again, we'll add them to the dataframe.

In [ ]:
commiseration_dyad_df['negative_word_similarity'] = [round(score, 2)
                                                     for score in commiseration_negative]

In [ ]:
commiseration_dyad_df['positive_word_similarity'] = [round(score, 2)
                                                     for score in commiseration_positive]

In [ ]:
celebration_dyad_df['negative_word_similarity'] = [round(score, 2)
                                                   for score in celebration_negative]

In [ ]:
celebration_dyad_df['positive_word_similarity'] = [round(score, 2)
                                                   for score in celebration_positive]

## Create unified dataset

Now that we've created separate subsets for each condition,
we'll combine them to create a single experiment dataframe.

In [ ]:
experiment_df = commiseration_dyad_df.append(celebration_dyad_df).reset_index(drop=True)

Let's say that we want to scale our positive and negative 
similarity scores, rather than using raw correlations.
We have a function that will help us (`scale`),
and we can get more information on how to use it by executing
this command:

In [ ]:
?scale

Let's scale the word similarity scores.

In [ ]:
experiment_df['negative_word_similarity_scaled'] = scale(experiment_df['negative_word_similarity'],
                                                 axis=0, with_mean=True, with_std=True, copy=True)

In [ ]:
experiment_df['positive_word_similarity_scaled'] = scale(experiment_df['positive_word_similarity'],
                                                 axis=0, with_mean=True, with_std=True, copy=True)

How do our data look now?

In [ ]:
experiment_df

## Save to file

In [ ]:
experiment_df.to_csv('../data/simulated_experiment_data.csv',
                     sep=',',
                     header=True,
                     index=False)